In [34]:
from transformers import BertModel, BertTokenizer
import re
tokenizer = BertTokenizer.from_pretrained("Rostlab/prot_bert", do_lower_case=False )
model = BertModel.from_pretrained("Rostlab/prot_bert")

/home/yisupeng/.conda/envs/py310/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [35]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]='3'

In [36]:
import torch

In [37]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# DEVICE = 'cpu'
DEVICE

device(type='cuda')

In [38]:
# Save the model's weights to a file
torch.save(model.state_dict(), "states/prot_bert_weights.pth")


In [39]:
type(model)

transformers.models.bert.modeling_bert.BertModel

In [40]:
tokenizer('A C D P', return_tensors='pt')

{'input_ids': tensor([[ 2,  6, 23, 14, 16,  3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1]])}

In [41]:
sample_enc = model(**tokenizer('A C D P', return_tensors='pt'))
sample_enc

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[ 0.0365,  0.1013,  0.0193,  ..., -0.0770, -0.0276,  0.0215],
         [ 0.1397,  0.0031,  0.0538,  ..., -0.0168, -0.0239, -0.0142],
         [ 0.1059,  0.0361,  0.0017,  ..., -0.0122, -0.0556,  0.0192],
         [ 0.0876,  0.0146,  0.0295,  ...,  0.0236,  0.0224, -0.0091],
         [ 0.1422,  0.0328, -0.0488,  ..., -0.0380,  0.0006,  0.0491],
         [ 0.0868,  0.0332,  0.0253,  ..., -0.0656, -0.0412,  0.0205]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[-0.2652,  0.2798, -0.2526,  ...,  0.2669,  0.2497, -0.2725]],
       grad_fn=<TanhBackward0>), hidden_states=None, past_key_values=None, attentions=None, cross_attentions=None)

In [42]:
sample_enc.last_hidden_state.shape

torch.Size([1, 6, 1024])

In [43]:
import pickle
ann_dataset = pickle.load(open('states/ann_dataset.pickle', 'rb'))

In [44]:
sample_data = ann_dataset.sample(1)
sample_data

,seq,ann
45689,M A A F L K M S V S V N F F R P F T R F L V P ...,GO:0005575 GO:0110165 GO:0032991


In [45]:
len(sample_data.iloc[0]['seq'])

805

In [46]:
sample_inputs = tokenizer(sample_data.iloc[0]['seq'], return_tensors='pt')['input_ids'].to(DEVICE)
sample_inputs

tensor([[ 2, 21,  6,  6, 19,  5, 12, 21, 10,  8, 10,  8, 17, 19, 19, 13, 16, 19,
         15, 13, 19,  5,  8, 16, 19, 15,  5, 22, 13, 12, 13, 17, 17,  5, 15, 11,
          5, 18, 13, 20, 21, 10, 10, 12, 11, 16,  6,  8, 15, 20, 16, 12, 17,  9,
         10, 15, 16, 16, 10,  9,  9,  5,  9,  5, 14, 12, 24, 12, 15, 15, 21, 12,
         10, 10,  8, 18,  9,  9, 23,  8, 10, 15, 11, 10, 10, 10, 12, 14,  9, 14,
         16,  5,  6,  6, 15, 13,  9, 19, 11,  9, 21, 24, 13,  5,  5,  7, 13,  9,
          8, 16,  9, 22, 11, 15,  9,  9,  9,  5, 12, 15,  5, 21,  9, 23,  8, 10,
         17, 15,  6, 12, 12, 12, 20,  5, 12, 20,  5, 20, 15, 12,  9, 12,  8, 12,
         12,  6, 13, 18, 11, 12, 12,  9, 21, 12,  6,  6,  6, 13,  9,  9,  6, 12,
         17, 11, 12,  5,  5,  9, 15, 15,  9,  9, 14, 12, 18, 12, 17, 19,  5, 19,
          5, 13,  5, 24, 14, 13, 17, 21, 14, 11,  6, 21,  7, 24, 12,  7,  6, 18,
          6, 21, 18, 19,  7, 18, 16,  5,  8, 19, 14, 21,  6, 20,  9, 17, 20, 21,
         12, 13, 12,  9,  5,

In [47]:
model.to(DEVICE)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30, 1024, padding_idx=0)
    (position_embeddings): Embedding(40000, 1024)
    (token_type_embeddings): Embedding(2, 1024)
    (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-29): 30 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=1024, out_features=1024, bias=True)
            (key): Linear(in_features=1024, out_features=1024, bias=True)
            (value): Linear(in_features=1024, out_features=1024, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=1024, out_features=1024, bias=True)
            (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.0, inpla

In [48]:
from transformers import AutoTokenizer, AutoModel, utils
from bertviz import model_view, head_view
utils.logging.set_verbosity_error()

inputs = tokenizer.encode(sample_data.iloc[0]['seq'][:10], return_tensors='pt').to(DEVICE)
outputs = model(inputs, output_attentions=True)
attention = outputs[-1]  # Output includes attention weights when output_attentions=True
tokens = tokenizer.convert_ids_to_tokens(inputs[0]) 

In [49]:
len(attention)

30

In [50]:
from bertviz import head_view
model_view(attention, tokens)

<IPython.core.display.Javascript object>

In [51]:
import sys
sys.platform

'linux'

In [52]:
#!pip install textpruner

  Using cached numpy-1.21.6-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.1 kB)
Using cached numpy-1.21.6-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (15.9 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llm2vec 0.2.2 requires peft, which is not installed.
contourpy 1.3.0 requires numpy>=1.23, but you have numpy 1.21.6 which is incompatible.
matplotlib 3.9.2 requires numpy>=1.23, but you have numpy 1.21.6 which is incompatible.
numexpr 2.10.1 requires numpy>=1.23.0, but you have numpy 1.21.6 which is incompatible.
pandas 2.2.2 requires numpy>=1.22.4; python_version < "3.11", but you have numpy 1.21.6 which is incompatible.
scipy 1.14.1 requires numpy<2.3,>=1.23.5, but y

In [53]:
import textpruner
from transformers import XLMRobertaForSequenceClassification,XLMRobertaTokenizer

from textpruner import summary, TransformerPruner
import sys


In [54]:
print("Before pruning:")
print(summary(model))

Before pruning:

LAYER NAME                  	        #PARAMS	     RATIO	 MEM(MB)
--model:                    	    419,931,136	   100.00%	 1601.91
  --embeddings:             	     40,994,816	     9.76%	  156.38
    --word_embeddings:      	         30,720	     0.01%	    0.12
    --position_embeddings:  	     40,960,000	     9.75%	  156.25
    --token_type_embeddings:	          2,048	     0.00%	    0.01
    --LayerNorm:            	          2,048	     0.00%	    0.01
  --encoder                 
    --layer:                	    377,886,720	    89.99%	 1441.52
  --pooler                  
    --dense:                	      1,049,600	     0.25%	    4.00



In [55]:
import numpy as np

In [56]:
pruner = TransformerPruner(model)

ffn_mask = textpruner.pruners.utils.random_mask_tensor((16,4096), p=0.25)
head_mask = textpruner.pruners.utils.random_mask_tensor((30,16), p=0.25, even_masks=False)
# ffn_mask = random_mask((16,4096))
# head_mask = random_mask((30,16))

ffn_mask, head_mask

(tensor([[0., 0., 0.,  ..., 1., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 1., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 1.],
         [0., 0., 0.,  ..., 1., 0., 0.],
         [1., 0., 0.,  ..., 0., 0., 0.]]),
 tensor([[1., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 1., 0., 0., 1., 1.],
         [0., 1., 1., 0., 1., 0., 1., 1., 0., 1., 1., 1., 1., 0., 1., 0.],
         [1., 1., 1., 1., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0.],
         [0., 0., 0., 1., 1., 0., 1., 0., 0., 1., 0., 1., 0., 1., 1., 0.],
         [0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 1.],
         [1., 0., 0., 0., 1., 1., 0., 0., 1., 1., 1., 0., 0., 1., 1., 0.],
         [1., 0., 1., 1., 0., 1., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0.],
         [1., 0., 0., 1., 0., 1., 0., 1., 0., 1., 1., 1., 1., 0., 0., 1.],
         [0., 1., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
         [0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 0., 0., 1

In [57]:
pruner.prune(head_mask=head_mask, ffn_mask=ffn_mask,save_model=True)

'./pruned_models/pruned_H7.633333333333334F1024'

In [59]:
print("After pruning:")
print(summary(model))

After pruning:

LAYER NAME                  	        #PARAMS	     RATIO	 MEM(MB)
--model:                    	    253,372,352	   100.00%	  966.54
  --embeddings:             	     40,994,816	    16.18%	  156.38
    --word_embeddings:      	         30,720	     0.01%	    0.12
    --position_embeddings:  	     40,960,000	    16.17%	  156.25
    --token_type_embeddings:	          2,048	     0.00%	    0.01
    --LayerNorm:            	          2,048	     0.00%	    0.01
  --encoder                 
    --layer:                	    211,327,936	    83.41%	  806.15
  --pooler                  
    --dense:                	      1,049,600	     0.41%	    4.00



In [60]:
pickle.dump(model, open('prot_bert_pruned_25.pickle', 'wb'))

In [21]:
import random

In [26]:
torch.save(model.state_dict(), "states/prot_bert_weights_pruned_25.pth")

In [33]:
model.load_state_dict(torch.load("states/prot_bert_weights_pruned_25.pth"))

<All keys matched successfully>

In [27]:
for i in range(16):
    print ((model.base_model.encoder.layer[i].intermediate.dense.weight.shape,
            model.base_model.encoder.layer[i].intermediate.dense.bias.shape,
            model.base_model.encoder.layer[i].attention.self.key.weight.shape))

(torch.Size([1024, 1024]), torch.Size([1024]), torch.Size([512, 1024]))
(torch.Size([1024, 1024]), torch.Size([1024]), torch.Size([384, 1024]))
(torch.Size([1024, 1024]), torch.Size([1024]), torch.Size([512, 1024]))
(torch.Size([1024, 1024]), torch.Size([1024]), torch.Size([384, 1024]))
(torch.Size([1024, 1024]), torch.Size([1024]), torch.Size([384, 1024]))
(torch.Size([1024, 1024]), torch.Size([1024]), torch.Size([576, 1024]))
(torch.Size([1024, 1024]), torch.Size([1024]), torch.Size([640, 1024]))
(torch.Size([1024, 1024]), torch.Size([1024]), torch.Size([512, 1024]))
(torch.Size([1024, 1024]), torch.Size([1024]), torch.Size([512, 1024]))
(torch.Size([1024, 1024]), torch.Size([1024]), torch.Size([384, 1024]))
(torch.Size([1024, 1024]), torch.Size([1024]), torch.Size([640, 1024]))
(torch.Size([1024, 1024]), torch.Size([1024]), torch.Size([640, 1024]))
(torch.Size([1024, 1024]), torch.Size([1024]), torch.Size([320, 1024]))
(torch.Size([1024, 1024]), torch.Size([1024]), torch.Size([384, 

In [ ]:
token =tokenizer(sample_data.iloc[0]['seq'],return_tensors="pt")

# model.device("cpu")
inference = textpruner.inference_time(model,token)
print(inference)

In [ ]:
# Save the model's weights to a file
torch.save(model.state_dict(), "states/prot_bert_weights_pruned.pth")


In [ ]:
!pip uninstall textpruner -y

In [61]:
!pip install "numpy<2.0" --upgrade

  Using cached numpy-1.26.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.2 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llm2vec 0.2.2 requires peft, which is not installed.
textpruner 1.1.post2 requires numpy<1.22, but you have numpy 1.26.4 which is incompatible.


In [29]:
model.config.save_pretrained('states/protbert_pruned')

In [30]:
model.config

BertConfig {
  "_name_or_path": "Rostlab/prot_bert",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.0,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 40000,
  "model_type": "bert",
  "num_attention_heads": 16,
  "num_hidden_layers": 30,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "pruned_heads": {
    "0": [
      0,
      5,
      9,
      10,
      11,
      12,
      13,
      15
    ],
    "1": [
      0,
      1,
      2,
      5,
      6,
      7,
      9,
      11,
      13,
      14
    ],
    "2": [
      0,
      5,
      6,
      9,
      10,
      11,
      14,
      15
    ],
    "3": [
      1,
      4,
      5,
      6,
      7,
      8,
      10,
      12,
      13,
      14
    ],
    "4": [
      0,
      1,
      3,
      7,
      8,
      